In [ ]:
import torch
import os

# Set the API key
os.environ['HF_TOKEN'] = ''

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

api_key = os.getenv('HF_TOKEN')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B",use_auth_token=api_key)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B",use_auth_token=api_key)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
import os
import pandas as pd
import re


df = pd.read_csv('/content/output_questions (2).tsv',sep='\t')
df = df.drop(['Correct Answer','Explanation'], axis=1)



# Function to prepare the prompt for the model
def prepare_prompt(question, options, prompt):
    options_text = "\n".join([f"Option {chr(65+i)}: {opt}" for i, opt in enumerate(options)])
    formatted_prompt = f"{prompt}\nQuestion: {question}\n{options_text}\nAnswer:"
    print("Formatted Prompt:\n", formatted_prompt)  # Debug print to see the formatted prompt
    return formatted_prompt

# Function to tokenize input
def tokenize_input(formatted_prompt, tokenizer):
    return tokenizer(formatted_prompt, return_tensors='pt')['input_ids']

# Function to generate response from the model
def generate_response(input_ids, model, tokenizer, max_length=512):
    response = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(response[0], skip_special_tokens=True)
    return generated_text

# Function to extract the answer from generated text
def extract_answer(generated_text):
    print("Generated text:", generated_text)  # Debug print to see the actual generated text
    # Find the last line that contains only a single uppercase letter
    match = re.findall(r'\b[A-D]\b', generated_text)
    return match[-1] if match else "No valid answer found"


# Number of rows to sample
n_samples = 100 # or any number of rows you want to sample

# Randomly sample rows from the DataFrame
sampled_df = df.sample(n=n_samples, random_state=42)
results = []
for _, row in sampled_df.iterrows():
    question = row['Question']
    options = [row['Option A'], row['Option B'], row['Option C'], row['Option D']]
    options_str = "\n".join([f"{chr(65 + i)}. {opt}" for i, opt in enumerate(options)])
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. "
        "You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset. "
        "Your task is to choose the best option among the four provided. Return your answer as a single uppercase letter: A, B, C, or D.\n\n"
        f"Question: {question}\n"
        f"Options:\n{options_str}"
    )


    # Prepare the prompt
    formatted_prompt = prepare_prompt(question, options, prompt)

    # Tokenize the input
    input_ids = tokenize_input(formatted_prompt, tokenizer)

    # Generate a response
    generated_text = generate_response(input_ids, model, tokenizer)

    # Extract the answer
    predicted_answer = extract_answer(generated_text)

    # Append results
    results.append({
        'Question': question,
        'Formatted Prompt': formatted_prompt,
        'Generated Text': generated_text,
        'llm38b': predicted_answer,
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('llm38b.csv', index=False)

print("Predicted answers saved to 'llm38b.csv'")


Formatted Prompt:
 You are a cybersecurity expert specializing in cyber threat intelligence. You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset. Your task is to choose the best option among the four provided. Return your answer as a single uppercase letter: A, B, C, or D.

Question: Which MITRE ATT&CK Enterprise technique involves an adversary using an existing, legitimate remote access application to maintain persistent access to a victim network?
Options:
A. T1190 - Exploit Public-Facing Application
B. T1071 - Application Layer Protocol
C. T1547.012 - Boot or Logon Autostart Execution
D. T1076 - Remote Desktop Protocol (RDP)
Question: Which MITRE ATT&CK Enterprise technique involves an adversary using an existing, legitimate remote access application to maintain persistent access to a victim network?
Option A: T1190 - Exploit Public-Facing Application
Option B: T1071 - Application Layer Protocol
Option C: T1547.012 - Boot

# New Section

In [6]:
import pandas as pd

cti=pd.read_csv('/content/output_questions (2).tsv', delimiter='\t')
answer=pd.read_csv('/content/llm38b.csv')
cti.head()


,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation
0,Which MITRE ATT&CK technique involves an adver...,Spearphishing Attachment (T1193),Social Engineering (T1140),Remote Access Tools (T1219),Man-in-the-Middle Attack (T1557),B,Social Engineering (T1140) is the correct answ...
1,Which MITRE ATT&CK technique involves an adver...,(A) Exploit Public-Facing Application (T1190),(B) Third-party Software (T1072),(C) Exploitation for Client Execution (T1203),(D) Exploitation for Privilege Escalation (T1068),B,Adversaries can exploit weaknesses in third-pa...
2,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Data Staged (T1074),Command and Control (T1009),Remote File Copy (T1105),C,Command and Control (T1009) corresponds to adv...
3,"In the context of MITRE ATT&CK for Enterprise,...",T1548.002 - Abuse Elevation Control Mechanism,T1059.001 - Command-Line Interface,T1566.001 - Phishing,T1078.002 - Valid Accounts,T1548.002 - Abuse Elevation Control Mechanism,Adversaries may abuse elevation control mechan...
4,Which MITRE ATT&CK technique involves an adver...,Process Injection (T1055) - Process Injection,Man-in-the-Middle (T1557) - Man-in-the-Middle,Control Device Identification (T818) - Control...,Manipulate Device Communication (T848) - Manip...,C,Manipulate Device Communication (T848) involve...


In [7]:
merged_df = pd.merge(cti, answer, on='Question', suffixes=('_gt', '_model'))
merged_df.head()




,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation,Formatted Prompt,Generated Text,llm38b
0,Which MITRE ATT&CK technique involves an adver...,Exfiltration Over Alternative Protocol (T1048),Remote Desktop Protocol (T1076),Packet Sniffing (T1040),Exploitation for Defense Evasion (T1211),C,Packet Sniffing (T1040) is the correct answer ...,You are a cybersecurity expert specializing in...,You are a cybersecurity expert specializing in...,C
1,"In the context of MITRE ATT&CK Enterprise, whi...",Account Discovery (T1087),External Remote Services (T1133),Valid Accounts (T1078),Inbox Rules (T1114),D,Inbox Rules (T1114) allows an adversary to man...,You are a cybersecurity expert specializing in...,You are a cybersecurity expert specializing in...,D
2,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Execution through API (T1106),Trusted Relationship (T1199),Trap (T1154),A,Process Injection (T1055) refers to an attacke...,You are a cybersecurity expert specializing in...,You are a cybersecurity expert specializing in...,D
3,Which MITRE ATT&CK technique involves an attac...,Process Injection - Dynamic-link Library Injec...,Valid Accounts (T1078),Trusted Relationship (T1199),Exploitation for Client Execution (T1203),B,Process Injection - Dynamic-link Library Injec...,You are a cybersecurity expert specializing in...,You are a cybersecurity expert specializing in...,C
4,Which MITRE ATT&CK technique involves an adver...,Initial Access: Spearphishing Attachment,Execution: Phishing,Defense Evasion: Spearphishing Link,Discovery: Phishing,A,The correct answer is Initial Access: Spearphi...,You are a cybersecurity expert specializing in...,You are a cybersecurity expert specializing in...,A


In [8]:
def compute_mcq_accuracy(df):
    correct = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row['llm38b']
        gt = row['Correct Answer']
        if pred in ['A', 'B', 'C', 'D', 'X']:
            total += 1
        else:
            print('Invalid response at row {}'.format(idx+1))
        if pred == gt:
            correct += 1
    return correct/total*100

In [9]:
print('Accuracy:', compute_mcq_accuracy(merged_df))

Accuracy: 38.31775700934579
